In [5]:
from pathlib import Path
import os
from extract_cosine_similarities import load_dict , load_word2vec_models, calculate_cosine_similarities
from calculate_embedding_bias import create_bias_dataframe
from plot_results import plot_results

import os
import pathlib

# Get the current notebook path
notebook_path = pathlib.Path.cwd()

# Create paths to the data and output folders
data_folder_path = notebook_path / '..' / '0_data'
output_folder_path = notebook_path / '..' / '2_pipeline' / 'preprocessed'
pipeline_out_folder_path = notebook_path / '..' / '2_pipeline' / 'out'
# Resolve and convert the paths to strings
data_folder_path = data_folder_path.resolve().as_posix()
output_folder_path = output_folder_path.resolve().as_posix()
foundations_path = os.path.join(data_folder_path, 'wordlist', 'dict_mft.json')
model_folder_path = os.path.join(data_folder_path, 'model')

# Targets (e.g., age groups)
age_groups_path = os.path.join(data_folder_path, 'wordlist', 'age_groups.json')

foundations = load_dict(foundations_path)
targets = load_dict(age_groups_path)

# models = load_word2vec_models(model_folder_path)

# # Change or add foundations to the dictionary here as needed
# # For example:
# # foundations['new_foundation_name'] = ['word1', 'word2', 'word3']

# # Calculate cosine similarities
# similarities = calculate_cosine_similarities(models, targets, foundations)

# # Calculate embedding bias
# colnames = ['year'] + [f'{target}_{foundation}' for foundation in foundations for target in targets]
# dfbias = pd.DataFrame(columns=colnames, index=range(1950, 2022))
# for year, yearly_similarities in similarities.items():
    # dfbias.loc[year] = [year] + [yearly_similarities[col] for col in colnames[1:]]

In [13]:
import pandas as pd
colnames = ['year'] + [f'{target}_{foundation}' for foundation in foundations for target in targets]
dfbias = pd.DataFrame(columns=colnames, index=range(1950, 2022))
for year, yearly_similarities in similarities.items():
   dfbias.loc[year] = [year] + [yearly_similarities[target][foundation] for col in colnames[1:] for target, foundation in [col.split('_', 1)]]




In [21]:
from extract_cosine_similarities import load_dict , load_word2vec_models, calculate_cosine_similarities
from calculate_embedding_bias import create_bias_dataframe


# data_folder_path = os.path.join(script_dir, '..', '0_data')
# pipeline_folder_path = os.path.join(script_dir, '..', '2_pipeline/preprocessed')
# tmp_folder_path = os.path.join(script_dir, '..', '2_pipeline/tmp')

# foundations_path = os.path.join(data_folder_path, 'wordlist', 'dict_mft.json')
posneg_path = os.path.join(data_folder_path, 'wordlist', 'dict_posneg.json')
scm_path = os.path.join(data_folder_path, 'wordlist', 'dict_scm.json')
# model_folder_path = os.path.join(data_folder_path, 'model')

# # Targets (e.g., age groups)
# age_groups_path = os.path.join(data_folder_path, 'wordlist', 'age_groups.json')

# foundations = load_dict(foundations_path)
posneg = load_dict(posneg_path)
scm = load_dict(scm_path)
targets = load_dict(age_groups_path)

models = load_word2vec_models(model_folder_path)

# Change or add foundations to the dictionary here as needed
# For example:
# foundations['new_foundation_name'] = ['word1', 'word2', 'word3']
evaluations = {'foundations': foundations,  'scm': scm,'posneg': posneg,}
for name, evaluations in evaluations.items():
    # Calculate cosine similarities
    similarities = calculate_cosine_similarities(models, targets, evaluations)

    # Calculate embedding bias
    dfbias = create_bias_dataframe(similarities, targets, evaluations)
    df_embedding_bias = calculate_embedding_bias(dfbias, targets, evaluations)

    # Save the DataFrame as a CSV
    csv_filepath = os.path.join(pipeline_folder_path, name+ '.csv')
    df_embedding_bias.to_csv(csv_filepath, index=False)
model_folder_path = os.path.join(data_folder_path, 'model')

# Targets (e.g., age groups)
age_groups_path = os.path.join(data_folder_path, 'wordlist', 'age_groups.json')

foundations = load_dict(foundations_path)
posneg = load_dict(posneg_path)
scm = load_dict(scm_path)
targets = load_dict(age_groups_path)

models = load_word2vec_models(model_folder_path)

# Change or add foundations to the dictionary here as needed
# For example:
# foundations['new_foundation_name'] = ['word1', 'word2', 'word3']
evaluations = {'scm': scm,'posneg': posneg,}
for name, evaluations in evaluations.items():
    # Calculate cosine similarities
    similarities = calculate_cosine_similarities(models, targets, evaluations)

    # Calculate embedding bias
    dfbias = create_bias_dataframe(similarities, targets, evaluations)
    
    # Save the DataFrame as a CSV
    csv_filepath = os.path.join(pipeline_folder_path, name+ '.csv')
    df_bias.to_csv(csv_filepath, index=False)

TypeError: 'dict_keys' object is not subscriptable

In [16]:
if any("_" in key for key in scm.keys()):
    print("True")

In [9]:
[yearly_similarities[col] for col in colnames[1:]]


KeyError: 'old_care_vir'

# granger

In [6]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VARResults
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.vector_ar.var_model import VARProcess

def granger_causality_matrix(data, variables, test='ssr_chi2test', verbose=False):
    X_train = pd.DataFrame(data)
    n = X_train.shape[1]
    granger_matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            data = X_train[[variables[i], variables[j]]]
            result = grangercausalitytests(data, maxlag=12, verbose=False)
            p_values = [round(result[i+1][0][test][1],4) for i in range(12)]
            if verbose:
                print(f'Y = {variables[i]}, X = {variables[j]}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            granger_matrix[i,j] = min_p_value
    return granger_matrix

def run_granger_test(df):
    model = VAR(df)
    results = model.fit(maxlags=15, ic='aic')
    lag_order = results.k_ar
    vecm_model = results.vecm(0)
    vecm_model.summary()
    vecm_model.test_granger_causality(caused='GDP', causing='indicoll', kind='f', signif=0.05).summary()


df=pd.read_csv(os.path.join(pipeline_out_folder_path, 'merged.csv'))
df.set_index('year', inplace=True)




In [9]:
!conda install pmdarima -c conda-forge -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/local/PSYCH-ADS/xuqian_chen/anaconda3/envs/jupyter_env/envs/ngram

  added / updated specs:
    - pmdarima


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    cython-0.29.33             |   py39h6a678d5_0         2.1 MB
    joblib-1.2.0               |     pyhd8ed1ab_0         205 KB  conda-forge
    libblas-3.9.0              |   12_linux64_mkl          12 KB  conda-forge
    libcblas-3.9.0             |   12_linux64_mkl          12 KB  conda-forge
    liblapack-3.9.0            |   12_linux64_mkl          12 KB  conda-forge
    numpy-1.22.3               |

In [19]:
import pmdarima as pm
from statsmodels.tsa.stattools import grangercausalitytests

model=pm.auto_arima(df['old_positive'],seasonal=False)
residuals=model.resid()
# concatenate residuals and independent variable and delete missing values

data = pd.concat([df['indi'],residuals])
granger_test=grangercausalitytests(data, maxlag=10, verbose=False)

print(granger_test)

ValueError: x contains NaN or inf values.

In [20]:
df['indi']

year
1950    0.000211
1951    0.000195
1952    0.000191
1953    0.000205
1954    0.000205
          ...   
2017    0.001363
2018    0.001330
2019         NaN
2020         NaN
2021         NaN
Name: indi, Length: 72, dtype: float64